## emp500_s5_labels.ipynb

Run this notebook after running:

* `emp500_s1_merge_sample_info.ipynb`
* `emp500_s2_add_prep_info_sample_names.ipynb`
* `emp500_s3_make_mapping_files_prep_info.ipynb`
* `emp500_s4_sample_summary.ipynb`

In [1]:
import pandas as pd

In [2]:
# df will be the list of samples we want to make labels for
df = pd.read_csv('/Users/luke.thompson/emp/500-metadata/labels/emp500_sample_summary.csv')
# remove studies: Kshtrika
df = df[df.emp500_principal_investigator != 'Kshtrika']
# idx is the list of per-study indexes (manually-generated)
idx = pd.read_excel('/Users/luke.thompson/emp/500-metadata/labels/emp500_per_study_indexes.xlsx')
idx = idx[['sample_name', 'study_index']]

In [3]:
# merge df with idx
df = pd.merge(df, idx, left_on='#SampleID', right_on='sample_name')
# remove samples without study IDs (negative and positive controls)
df = df[df.emp500_study_id != 'Not applicable']

In [4]:
# split PI-study and sample name to print on barcode
df['pi_study'] = [x.split('.', maxsplit=1)[0] for x in df['#SampleID']]
df['sample_name_only'] = [x.split('.', maxsplit=1)[1] for x in df['#SampleID']]

In [5]:
# convert study IDs to integers, sort by study_id and index, and get list of unique ones
df.emp500_study_id = [int(x) for x in df.emp500_study_id]
df.sort_values(['emp500_study_id', 'study_index'], ascending=True, inplace=True)
studies = sorted(list(set(df.emp500_study_id)))

In [6]:
# df2 is the table of labels/aliquots (9 for each sample)
# dfs is a subset of df just for each study

# master label list (df2)
df2 = pd.DataFrame(columns=['barcode1', 'barcode2', 'project', 'study_number', 'principal_investigator', 
                            'study_title', 'sample_name', 'sample_number', 'aliquot_number', 'empo_3', 
                            'study_num_pi'], 
                   index=range(df.shape[0]*9))
i = 0
for study in studies:
    dfs = df[df.emp500_study_id == study]
    for index, sample in dfs.iterrows():
        for k in range(1,10):
            barcode = '%s.%s.s%s.a%s' % (sample.emp500_principal_investigator.lower(),
                                       str(study), 
                                       str(sample.study_index).zfill(3), 
                                       str(k).zfill(2))
            df2.iloc[i]['barcode1'] = barcode
            df2.iloc[i]['barcode2'] = barcode
            df2.iloc[i]['project'] = 'EMP500'
            df2.iloc[i]['study_number'] = sample.emp500_study_id
            df2.iloc[i]['principal_investigator'] = sample.emp500_principal_investigator
            df2.iloc[i]['study_title'] = sample.emp500_title
            df2.iloc[i]['sample_name'] = sample.sample_name_only
            df2.iloc[i]['sample_number'] = sample.study_index
            df2.iloc[i]['aliquot_number'] = k
            df2.iloc[i]['empo_3'] = sample.empo_3
            df2.iloc[i]['study_num_pi'] = '%s-%s' % (sample.emp500_study_id, sample.emp500_principal_investigator)
            i += 1
df2.to_excel('/Users/luke.thompson/emp/500-metadata/labels/emp500_labels.xlsx', index=None)

In [7]:
# df2 is the table of labels/aliquots (2 for each sample)
# dfs is a subset of df just for each study

# master label list (df2)
df2 = pd.DataFrame(columns=['barcode1', 'barcode2', 'project', 'study_number', 'principal_investigator', 
                            'study_title', 'sample_name', 'sample_number', 'aliquot_number', 'empo_3',
                            'study_num_pi'], 
                   index=range(df.shape[0]*9))
i = 0
for study in studies:
    dfs = df[df.emp500_study_id == study]
    for index, sample in dfs.iterrows():
        for k in range(10,12):
            barcode = '%s.%s.s%s.a%s' % (sample.emp500_principal_investigator.lower(),
                                       str(study), 
                                       str(sample.study_index).zfill(3), 
                                       str(k).zfill(2))
            df2.iloc[i]['barcode1'] = barcode
            df2.iloc[i]['barcode2'] = barcode
            df2.iloc[i]['project'] = 'EMP500'
            df2.iloc[i]['study_number'] = sample.emp500_study_id
            df2.iloc[i]['principal_investigator'] = sample.emp500_principal_investigator
            df2.iloc[i]['study_title'] = sample.emp500_title
            df2.iloc[i]['sample_name'] = sample.sample_name_only
            df2.iloc[i]['sample_number'] = sample.study_index
            df2.iloc[i]['aliquot_number'] = k
            df2.iloc[i]['empo_3'] = sample.empo_3
            df2.iloc[i]['study_num_pi'] = '%s-%s' % (sample.emp500_study_id, sample.emp500_principal_investigator)
            i += 1
df2.to_excel('/Users/luke.thompson/emp/500-metadata/labels/emp500_labels_extra.xlsx', index=None)

In [8]:
# gsheet is the excel files of combined worksheets
# dfi is the per-study table of labels/aliquots
# dfs is a subset of df just for each study

# master label list (dfi -> gsheet)
gsheet = pd.ExcelWriter('/Users/luke.thompson/emp/500-metadata/labels/emp500_gsheet.xlsx')
for study in studies:
    i = 0
    dfs = df[df.emp500_study_id == study]
    dfi = pd.DataFrame(columns=['sample_name', 'sample_number', 'aliquot_number', 'empo_3', 'barcode', 'scanned_barcode', 'notes'], 
                       index=range(dfs.shape[0]*9))
    for index, sample in dfs.iterrows():
        for k in range(1,10):
            barcode = '%s.%s.s%s.a%s' % (sample.emp500_principal_investigator.lower(),
                                       str(study), 
                                       str(sample.study_index).zfill(3), 
                                       str(k).zfill(2))
            dfi.iloc[i]['sample_name'] = sample.sample_name_only
            dfi.iloc[i]['sample_number'] = sample.study_index
            dfi.iloc[i]['aliquot_number'] = k
            dfi.iloc[i]['empo_3'] = sample.empo_3
            dfi.iloc[i]['barcode'] = barcode
            dfi.iloc[i]['scanned_barcode'] = ''
            dfi.iloc[i]['notes'] = ''
            i += 1
    dfi.to_excel(gsheet, '%s-%s' % (str(study), sample.emp500_principal_investigator))
gsheet.save()

In [9]:
# gsheet is the excel files of combined worksheets
# dfi is the per-study table of labels/aliquots
# dfs is a subset of df just for each study

# master label list (dfi -> gsheet)
gsheet = pd.ExcelWriter('/Users/luke.thompson/emp/500-metadata/labels/emp500_gsheet_extra.xlsx')
for study in studies:
    i = 0
    dfs = df[df.emp500_study_id == study]
    dfi = pd.DataFrame(columns=['sample_name', 'sample_number', 'aliquot_number', 'empo_3', 'barcode', 'scanned_barcode', 'notes'], 
                       index=range(dfs.shape[0]*9))
    for index, sample in dfs.iterrows():
        for k in range(10,12):
            barcode = '%s.%s.s%s.a%s' % (sample.emp500_principal_investigator.lower(),
                                       str(study), 
                                       str(sample.study_index).zfill(3), 
                                       str(k).zfill(2))
            dfi.iloc[i]['sample_name'] = sample.sample_name_only
            dfi.iloc[i]['sample_number'] = sample.study_index
            dfi.iloc[i]['aliquot_number'] = k
            dfi.iloc[i]['empo_3'] = sample.empo_3
            dfi.iloc[i]['barcode'] = barcode
            dfi.iloc[i]['scanned_barcode'] = ''
            dfi.iloc[i]['notes'] = ''
            i += 1
    dfi.to_excel(gsheet, '%s-%s' % (str(study), sample.emp500_principal_investigator))
gsheet.save()

In [10]:
# dfbox is per-box per-study labels
# dfs is a subset of df just for each study

dfbox = pd.DataFrame(columns=['col1', 'col2', 'col3', 'col4'], 
                     index=range(len(studies)))
i = 0
for study in studies:
    dfs = df[df.emp500_study_id == study]
    example = dfs.iloc[0]
    trim = 28
    label = '%s\r%s\r%s\r' % (
        ('EMP500 '+str(example['emp500_study_id'])+'-'+example['emp500_principal_investigator'])[:trim],
        example['emp500_title'][:trim],
        (example['empo_3']+' / '+example['sampling_method'])[:trim])
    dfbox.iloc[i]['col1'] = label + 'Box 1 of __'
    dfbox.iloc[i]['col2'] = label + 'Box 2 of __'
    dfbox.iloc[i]['col3'] = label + 'Box 3 of __'
    dfbox.iloc[i]['col4'] = label + 'Box 4 of __'
    i += 1
dfbox.to_excel('/Users/luke.thompson/emp/500-metadata/labels/emp500_box_labels.xlsx', index=None)